<h2 align='center' style='color:blue'>Campaign Performance Analysis - Nova Mart</h2>

# import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Datas

## dim_campaigns

In [2]:
df_campaigns_1 = pd.read_csv('dim_campaigns.csv')
df_campaigns_1.head(3)

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [3]:
df_campaigns_1.shape

(2, 4)

## dim_products

In [4]:
df_products_1 = pd.read_csv('dim_products.csv')
df_products_1.head()

,product_code,product_name,category
0,P01,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
2,P03,Atliq_Suflower_Oil (1L),Grocery & Staples
3,P04,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
4,P05,Atliq_Scrub_Sponge_For_Dishwash,Home Care


In [5]:
df_products_1.shape

(15, 3)

## dim_stores

In [6]:
df_dim_stores_1 = pd.read_csv('dim_stores.csv')
df_dim_stores_1.head()

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam
4,STCBE-3,Coimbatore


In [7]:
df_dim_stores_1.shape

(50, 2)

### fact_events

In [8]:
df_fact_events_1 = pd.read_csv('fact_events.csv')
df_fact_events_1.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [9]:
df_fact_events_1.shape

(1510, 9)

# Data Cleaning

## Removes Duplicates

In [10]:
# Count duplicates based on store_id, campaign_id, and product_code
num_duplicates = df_fact_events_1.duplicated(subset=['store_id', 'campaign_id', 'product_code']).sum()

# Remove the duplicate rows
df_fact_events_2 = df_fact_events_1.drop_duplicates(subset=['store_id', 'campaign_id', 'product_code']).copy()

print(f"Removed {num_duplicates} duplicate rows.")

Removed 10 duplicate rows.


## Removes Null

In [11]:
df_fact_events_2.isnull().sum()

event_id                        0
store_id                        0
campaign_id                     0
product_code                    0
base_price(before_promo)        0
quantity_sold(before_promo)    20
promo_type                      0
base_price(after_promo)         0
quantity_sold(after_promo)      0
dtype: int64

In [45]:
df_fact_events_2['quantity_sold(before_promo)'].median()

np.float64(78.0)

In [12]:
df_fact_null = df_fact_events_2.loc[df_fact_events_2['quantity_sold(before_promo)'].isna()==True]
df_fact_null.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
63,70c312,STHYD-4,CAMP_SAN_01,P13,350,NaN,BOGOF,175,534
119,d31787,STMYS-2,CAMP_SAN_01,P10,50,NaN,25% OFF,37,20
141,141d98,STCHE-4,CAMP_SAN_01,P03,200,NaN,BOGOF,100,1695
163,873333,STMLR-0,CAMP_DIW_01,P15,3000,NaN,500 Cashback,2500,509
341,2ef46d,STMDU-0,CAMP_DIW_01,P02,860,NaN,33% OFF,576,430


In [13]:
df_fact_null.index

Index([  63,  119,  141,  163,  341,  391,  558,  714,  745,  758,  924,  967,
       1035, 1072, 1163, 1164, 1196, 1247, 1419, 1507],
      dtype='int64')

In [14]:
df_fact_events_2.loc[df_fact_events_2.product_code=='P13'].sample(5)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
879,f7aec5,STCHE-5,CAMP_SAN_01,P13,350,87.0,BOGOF,175,341
1070,b8269b,STBLR-1,CAMP_DIW_01,P13,350,77.0,BOGOF,175,232
199,60ba75,STVJD-0,CAMP_SAN_01,P13,350,60.0,BOGOF,175,234
1236,407f1c,STBLR-2,CAMP_SAN_01,P13,350,117.0,BOGOF,175,469
685,f940cf,STHYD-6,CAMP_DIW_01,P13,350,77.0,BOGOF,175,260


In [15]:
df_prod_prom = df_fact_events_2.groupby(['product_code','promo_type'])['quantity_sold(before_promo)'].agg('median').round()
df_prod_prom.head()

product_code  promo_type
P01           33% OFF       272.0
P02           33% OFF       366.0
P03           25% OFF       314.0
              BOGOF         331.0
P04           25% OFF       291.0
Name: quantity_sold(before_promo), dtype: float64

In [16]:
def qty_sold_null(row):
    if pd.isnull(row['quantity_sold(before_promo)']):
        return df_prod_prom.loc[row['product_code'], row['promo_type']]
    else:
        return row['quantity_sold(before_promo)']

In [17]:
df_fact_events_2['quantity_sold(before_promo)'] = df_fact_events_2.apply(qty_sold_null, axis=1)

In [18]:
df_fact_events_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


# Exploratory Data Analysis

## How many cities have more than 5 stores?

In [44]:
result = df_dim_stores_1['city'].value_counts().loc[lambda x: x > 5]
result

city
Bengaluru    10
Chennai       8
Hyderabad     7
Name: count, dtype: int64

## Total Quantity Sold BOGOF during Diwali

### Merge all dataset

#### event x store

In [20]:
df_merge_1 = pd.merge(df_fact_events_2,df_dim_stores_1,on='store_id')
df_merge_1.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Chennai


#### event x product 

In [21]:
df_merge_1 = pd.merge(df_merge_1,df_products_1,on='product_code')
df_merge_1.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,product_name,category
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples


#### event x campaigns

In [22]:
df_merge_1 = pd.merge(df_merge_1,df_campaigns_1,on='campaign_id').drop('campaign_id',axis=1)
df_merge_1.head(2)

,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,product_name,category,campaign_name,start_date,end_date
0,8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12-11-2023,18-11-2023
1,20618e,STCHE-3,P04,370,379.0,BOGOF,185,1622,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10-01-2024,16-01-2024


#### reset_index

In [23]:
df_merge_1 = df_merge_1.reset_index(drop=True).set_index('event_id')
df_merge_1.head(2)

,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,product_name,category,campaign_name,start_date,end_date
event_id,,,,,,,,,,,,,
8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12-11-2023,18-11-2023
20618e,STCHE-3,P04,370,379.0,BOGOF,185,1622,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10-01-2024,16-01-2024


#### cast to datetime

In [24]:
df_merge_1 = df_merge_1.astype({
    'start_date': 'datetime64[ns]',
    'end_date': 'datetime64[ns]'
})
df_merge_1.head()

,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,product_name,category,campaign_name,start_date,end_date
event_id,,,,,,,,,,,,,
8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,2023-12-11,2023-11-18
20618e,STCHE-3,P04,370,379.0,BOGOF,185,1622,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,2024-10-01,2024-01-16
f30579,STBLR-9,P02,860,337.0,33% OFF,576,488,Bengaluru,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,2023-12-11,2023-11-18
4f570c,STBLR-7,P05,55,122.0,25% OFF,41,107,Bengaluru,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,2023-12-11,2023-11-18
6d153f,STHYD-5,P15,3000,122.0,500 Cashback,2500,272,Hyderabad,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,2024-10-01,2024-01-16


### Total Qty Sold

In [25]:
tqs = df_merge_1.loc[(df_merge_1.promo_type == 'BOGOF') & (df_merge_1.campaign_name == 'Diwali'),'quantity_sold(after_promo)'].sum()
tqs

np.int64(34461)

## Best Store Performance

In [50]:
df_merge_1.loc[df_merge_1.campaign_name=='Diwali'].groupby('store_id')['quantity_sold(after_promo)'].agg('sum')

store_id
STBLR-0    4759
STBLR-1    4146
STBLR-2    4169
STBLR-3    4373
STBLR-4    4408
STBLR-5    4193
STBLR-6    4857
STBLR-7    4893
STBLR-8    4395
STBLR-9    4186
STCBE-0    3381
STCBE-1    2942
STCBE-2    3093
STCBE-3    3077
STCBE-4    2907
STCHE-0    4100
STCHE-1    3595
STCHE-2    4093
STCHE-3    4605
STCHE-4    5013
STCHE-5    4052
STCHE-6    4445
STCHE-7    4779
STHYD-0    4460
STHYD-1    3778
STHYD-2    4266
STHYD-3    4272
STHYD-4    4227
STHYD-5    4346
STHYD-6    4153
STMDU-0    3545
STMDU-1    3266
STMDU-2    2981
STMDU-3    3071
STMLR-0    2027
STMLR-1    2151
STMLR-2    2138
STMYS-0    3543
STMYS-1    4779
STMYS-2    4130
STMYS-3    4347
STTRV-0    2180
STTRV-1    2072
STVJD-0    2392
STVJD-1    2312
STVSK-0    3005
STVSK-1    3078
STVSK-2    2860
STVSK-3    2656
STVSK-4    2908
Name: quantity_sold(after_promo), dtype: int64

## Best Campaign

### groupby campaign_name and promo_type

In [27]:
df_best_campaign = df_merge_1.groupby(['campaign_name','promo_type'])[['quantity_sold(before_promo)',
                                                                    'quantity_sold(after_promo)',
                                                                       ]].agg('sum')
df_best_campaign

quantity_sold(before_promo)  \
campaign_name promo_type                                  
Diwali        25% OFF                           37479.0   
              33% OFF                           29240.0   
              50% OFF                           16873.0   
              500 Cashback                      16513.0   
              BOGOF                             10018.0   
Sankranti     25% OFF                            6501.0   
              33% OFF                           34300.0   
              50% OFF                            4440.0   
              500 Cashback                       5573.0   
              BOGOF                             48045.0   

                            quantity_sold(after_promo)  
campaign_name promo_type                                
Diwali        25% OFF                            32983  
              33% OFF                            43117  
              50% OFF                            22074  
              500 Cashback                       50769  
              BOGOF                              34461  
Sankranti     25% OFF                             5307  
              33% OFF                            47459  
              50% OFF                             6100  
              500 Cashback                       12411  
              BOGOF                             180792

### add diff column

In [28]:
df_best_campaign['diff'] = df_best_campaign['quantity_sold(after_promo)'] - df_best_campaign['quantity_sold(before_promo)']
df_best_campaign

quantity_sold(before_promo)  \
campaign_name promo_type                                  
Diwali        25% OFF                           37479.0   
              33% OFF                           29240.0   
              50% OFF                           16873.0   
              500 Cashback                      16513.0   
              BOGOF                             10018.0   
Sankranti     25% OFF                            6501.0   
              33% OFF                           34300.0   
              50% OFF                            4440.0   
              500 Cashback                       5573.0   
              BOGOF                             48045.0   

                            quantity_sold(after_promo)      diff  
campaign_name promo_type                                          
Diwali        25% OFF                            32983   -4496.0  
              33% OFF                            43117   13877.0  
              50% OFF                            22074    5201.0  
              500 Cashback                       50769   34256.0  
              BOGOF                              34461   24443.0  
Sankranti     25% OFF                             5307   -1194.0  
              33% OFF                            47459   13159.0  
              50% OFF                             6100    1660.0  
              500 Cashback                       12411    6838.0  
              BOGOF                             180792  132747.0

### find the best campaign and what event

In [29]:
# Get the index (a tuple) of the row with the maximum diff value
best_idx = df_best_campaign['diff'].idxmax()

# Unpack the tuple (campaign_name, promo_type)
best_campaign, best_promo = best_idx

print("Best campaign:", best_campaign)
print("Best promo type:", best_promo)

Best campaign: Sankranti
Best promo type: BOGOF


## Highest Revenue Percentage (IR%) product during Sankranti

Incremental Revenue Percentage (IR%) is calculated as:

$$
\text{IR\%} = \left(\frac{R_{\text{revenue after promo}} - R_{\text{revenue before promo}}}{R_{\text{revenue before promo}}}\right) \times 100
$$

In [30]:
df_merge_1.head(1)

,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,product_name,category,campaign_name,start_date,end_date
event_id,,,,,,,,,,,,,
8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,2023-12-11,2023-11-18


In [31]:
df_sakranti = df_merge_1.loc[df_merge_1.campaign_name=='Sankranti',['product_code','promo_type','product_name','base_price(before_promo)','quantity_sold(before_promo)','base_price(after_promo)','quantity_sold(after_promo)']]
df_sakranti.head(2)

,product_code,promo_type,product_name,base_price(before_promo),quantity_sold(before_promo),base_price(after_promo),quantity_sold(after_promo)
event_id,,,,,,,
20618e,P04,BOGOF,Atliq_Farm_Chakki_Atta (1KG),370,379.0,185,1622
6d153f,P15,500 Cashback,Atliq_Home_Essential_8_Product_Combo,3000,122.0,2500,272


In [32]:
df_sakranti['revenue(before_promo)']=df_sakranti['base_price(before_promo)'] * df_sakranti['quantity_sold(before_promo)']
df_sakranti['revenue(after_promo)']=df_sakranti['base_price(after_promo)'] * df_sakranti['quantity_sold(after_promo)']

df_sakranti.head(2).T

event_id,20618e,6d153f
product_code,P04,P15
promo_type,BOGOF,500 Cashback
product_name,Atliq_Farm_Chakki_Atta (1KG),Atliq_Home_Essential_8_Product_Combo
base_price(before_promo),370,3000
quantity_sold(before_promo),379.0,122.0
base_price(after_promo),185,2500
quantity_sold(after_promo),1622,272
revenue(before_promo),140230.0,366000.0
revenue(after_promo),300070,680000


In [33]:
df_ir = df_sakranti.groupby('product_code').agg({
    'product_name': 'first',
    'revenue(before_promo)': 'sum',
    'revenue(after_promo)': 'sum'
})
df_ir

,product_name,revenue(before_promo),revenue(after_promo)
product_code,,,
P01,Atliq_Masoor_Dal (1KG),2175112.0,2011235
P02,Atliq_Sonamasuri_Rice (10KG),18622440.0,17262720
P03,Atliq_Suflower_Oil (1L),3240200.0,6118500
P04,Atliq_Farm_Chakki_Atta (1KG),6813550.0,12779800
P05,Atliq_Scrub_Sponge_For_Dishwash,59675.0,35875
P06,Atliq_Fusion_Container_Set_of_3,561080.0,342722
P07,Atliq_Curtains,573600.0,1065450
P08,Atliq_Double_Bedsheet_set,2485910.0,4649330
P09,Atliq_Body_Milk_Nourishing_Lotion (120ML),239040.0,146462


In [34]:
df_ir['ir_percentage'] = (df_ir['revenue(after_promo)'] - df_ir['revenue(before_promo)'])/df_ir['revenue(before_promo)']*100
df_ir.sort_values('ir_percentage',ascending=False)

,product_name,revenue(before_promo),revenue(after_promo),ir_percentage
product_code,,,,
P13,Atliq_High_Glo_15W_LED_Bulb,1741250.0,3303125,89.698492
P03,Atliq_Suflower_Oil (1L),3240200.0,6118500,88.830936
P14,Atliq_waterproof_Immersion_Rod,4542060.0,8534850,87.907029
P04,Atliq_Farm_Chakki_Atta (1KG),6813550.0,12779800,87.564485
P08,Atliq_Double_Bedsheet_set,2485910.0,4649330,87.027286
P07,Atliq_Curtains,573600.0,1065450,85.747908
P15,Atliq_Home_Essential_8_Product_Combo,16719000.0,31027500,85.582272
P02,Atliq_Sonamasuri_Rice (10KG),18622440.0,17262720,-7.301514
P01,Atliq_Masoor_Dal (1KG),2175112.0,2011235,-7.534187


In [35]:
# Get the index (a tuple) of the row with the maximum diff value
best_ir_prod = df_ir['ir_percentage'].idxmax()

# print("Best product:", best_product)
df_ir.loc[df_ir.index == best_ir_prod]

,product_name,revenue(before_promo),revenue(after_promo),ir_percentage
product_code,,,,
P13,Atliq_High_Glo_15W_LED_Bulb,1741250.0,3303125,89.698492


## Lowest Store Incremental Sold Unit Percentage (ISU%) during Diwali Campaign

Sold Units Percentage (ISU%) measures the percentage change in the number of units sold due to a promotion, relative to the baseline (units sold before the promotion). In other words, it quantifies how much the promotion increased (or decreased) the number of units sold.

The formula is given by:
$$
\text{ISU\%} = \left(\frac{Q_{\text{unit sold after promotion}} - Q_{\text{unit sold before promotion}}}{Q_{\text{unit sold before promotion}}}\right) \times 100
$$

In [36]:
df_merge_1.head(1).T

event_id,8481be
store_id,STCHE-1
product_code,P04
base_price(before_promo),290
quantity_sold(before_promo),327.0
promo_type,25% OFF
base_price(after_promo),217
quantity_sold(after_promo),287
city,Chennai
product_name,Atliq_Farm_Chakki_Atta (1KG)
category,Grocery & Staples


In [37]:
df_diwali_visa = df_merge_1.loc[(df_merge_1.campaign_name == 'Diwali') & (df_merge_1.city=='Visakhapatnam'),
                        ['store_id','product_code','product_name','quantity_sold(before_promo)','quantity_sold(after_promo)']]
df_diwali_visa.head(1).T

event_id,ba86f4
store_id,STVSK-1
product_code,P13
product_name,Atliq_High_Glo_15W_LED_Bulb
quantity_sold(before_promo),61.0
quantity_sold(after_promo),204


In [38]:
df_isu = df_diwali_visa.groupby(['store_id'])[['quantity_sold(before_promo)','quantity_sold(after_promo)']].agg('sum')
df_isu

,quantity_sold(before_promo),quantity_sold(after_promo)
store_id,,
STVSK-0,1757.0,3005
STVSK-1,1903.0,3078
STVSK-2,1701.0,2860
STVSK-3,1780.0,2656
STVSK-4,1926.0,2908


In [39]:
df_isu['isu_percent'] = (df_isu['quantity_sold(after_promo)'] - df_isu['quantity_sold(before_promo)'])/df_isu['quantity_sold(before_promo)']*100
df_isu = df_isu.sort_values('isu_percent',ascending=True)
df_isu.head()

,quantity_sold(before_promo),quantity_sold(after_promo),isu_percent
store_id,,,
STVSK-3,1780.0,2656,49.213483
STVSK-4,1926.0,2908,50.986501
STVSK-1,1903.0,3078,61.744614
STVSK-2,1701.0,2860,68.136390
STVSK-0,1757.0,3005,71.030165


In [40]:
# Get the index (a tuple) of the row with the maximum diff value
worst_isu_store = df_isu['isu_percent'].idxmin()
worst_isu_store
# print("Best product:", best_product)
df_isu.loc[df_isu.index == worst_isu_store]

,quantity_sold(before_promo),quantity_sold(after_promo),isu_percent
store_id,,,
STVSK-3,1780.0,2656,49.213483


## Negative Promo (based on ISU & IR) during Sankranti

In [41]:
df_sak = df_sakranti.groupby(['product_code','promo_type'])[['quantity_sold(before_promo)','revenue(before_promo)','quantity_sold(after_promo)','revenue(after_promo)']].agg('sum').reset_index()
df_sak

,product_code,promo_type,quantity_sold(before_promo),revenue(before_promo),quantity_sold(after_promo),revenue(after_promo)
0,P01,33% OFF,12646.0,2175112.0,17489,2011235
1,P02,33% OFF,21654.0,18622440.0,29970,17262720
2,P03,BOGOF,16201.0,3240200.0,61185,6118500
3,P04,BOGOF,18415.0,6813550.0,69080,12779800
4,P05,25% OFF,1085.0,59675.0,875,35875
5,P06,25% OFF,1352.0,561080.0,1102,342722
6,P07,BOGOF,1912.0,573600.0,7103,1065450
7,P08,BOGOF,2089.0,2485910.0,7814,4649330
8,P09,25% OFF,2656.0,239040.0,2186,146462
9,P10,25% OFF,1408.0,70400.0,1144,42328


In [42]:
df_sak['ir_percent'] = (df_sak['revenue(after_promo)']-df_sak['revenue(before_promo)'])/df_sak['revenue(before_promo)']*100
df_sak['isu_percent'] = (df_sak['quantity_sold(after_promo)']-df_sak['quantity_sold(before_promo)'])/df_sak['quantity_sold(before_promo)']*100

df_sak

,product_code,promo_type,quantity_sold(before_promo),revenue(before_promo),quantity_sold(after_promo),revenue(after_promo),ir_percent,isu_percent
0,P01,33% OFF,12646.0,2175112.0,17489,2011235,-7.534187,38.296695
1,P02,33% OFF,21654.0,18622440.0,29970,17262720,-7.301514,38.403990
2,P03,BOGOF,16201.0,3240200.0,61185,6118500,88.830936,277.661873
3,P04,BOGOF,18415.0,6813550.0,69080,12779800,87.564485,275.128971
4,P05,25% OFF,1085.0,59675.0,875,35875,-39.882698,-19.354839
5,P06,25% OFF,1352.0,561080.0,1102,342722,-38.917445,-18.491124
6,P07,BOGOF,1912.0,573600.0,7103,1065450,85.747908,271.495816
7,P08,BOGOF,2089.0,2485910.0,7814,4649330,87.027286,274.054572
8,P09,25% OFF,2656.0,239040.0,2186,146462,-38.729083,-17.695783
9,P10,25% OFF,1408.0,70400.0,1144,42328,-39.875000,-18.750000


In [43]:
df_sak.loc[(df_sak['ir_percent'] < 0) & (df_sak['isu_percent'] < 0)]

,product_code,promo_type,quantity_sold(before_promo),revenue(before_promo),quantity_sold(after_promo),revenue(after_promo),ir_percent,isu_percent
4,P05,25% OFF,1085.0,59675.0,875,35875,-39.882698,-19.354839
5,P06,25% OFF,1352.0,561080.0,1102,342722,-38.917445,-18.491124
8,P09,25% OFF,2656.0,239040.0,2186,146462,-38.729083,-17.695783
9,P10,25% OFF,1408.0,70400.0,1144,42328,-39.875000,-18.750000


In [46]:
df_merge_1.head(1).T

event_id,8481be
store_id,STCHE-1
product_code,P04
base_price(before_promo),290
quantity_sold(before_promo),327.0
promo_type,25% OFF
base_price(after_promo),217
quantity_sold(after_promo),287
city,Chennai
product_name,Atliq_Farm_Chakki_Atta (1KG)
category,Grocery & Staples


In [47]:
df_merge_1.groupby('category')['base_price(before_promo)'].agg('min')

category
Combo1               3000
Grocery & Staples     156
Home Appliances       350
Home Care              55
Personal Care          50
Name: base_price(before_promo), dtype: int64